In [44]:
!pip install keras
!pip install tensorflow

In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [46]:
from numpy import array

In [59]:
def split_sequence(sequence,n_steps):
    x,y=list(),list()
    for i in range (len(sequence)):
        end_ix=i+n_steps
        if end_ix>len(sequence)-1:
            break
        seq_x,seq_y=sequence[i:end_ix],sequence[end_ix]
        x.append(seq_x)
        y.append(seq_y)
    return array(x),array(y)


#day wise cashflow in an organisation
raw_seq=[2163480,3362496,3247884,5631955,5501340,3422760,3197472,2127160,2002725]
n_steps=3
x,y=split_sequence(raw_seq,n_steps)
for i in range(len(x)):
    print(x[i],y[i])

[2163480 3362496 3247884] 5631955
[3362496 3247884 5631955] 5501340
[3247884 5631955 5501340] 3422760
[5631955 5501340 3422760] 3197472
[5501340 3422760 3197472] 2127160
[3422760 3197472 2127160] 2002725


In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional

In [60]:
raw_seq=[2163480,3362496,3247884,5631955,5501340,3422760,3197472,2127160,2002725]
n_steps=3
x,y=split_sequence(raw_seq,n_steps)

In [61]:
x

array([[2163480, 3362496, 3247884],
       [3362496, 3247884, 5631955],
       [3247884, 5631955, 5501340],
       [5631955, 5501340, 3422760],
       [5501340, 3422760, 3197472],
       [3422760, 3197472, 2127160]])

In [62]:
y

array([5631955, 5501340, 3422760, 3197472, 2127160, 2002725])

In [63]:
n_features=1
x=x.reshape((x.shape[0],x.shape[1],n_features))

In [64]:
x

array([[[2163480],
        [3362496],
        [3247884]],

       [[3362496],
        [3247884],
        [5631955]],

       [[3247884],
        [5631955],
        [5501340]],

       [[5631955],
        [5501340],
        [3422760]],

       [[5501340],
        [3422760],
        [3197472]],

       [[3422760],
        [3197472],
        [2127160]]])

In [65]:
model1=Sequential()
model1.add(LSTM(50,activation='relu',input_shape=(n_steps,n_features)))
model1.add(Dense(1))
model1.compile(optimizer='adam',loss='mse')
model1.fit(x,y,epochs=200,verbose=0)

In [69]:
x_input=array([5501340, 3422760, 3197472])
x_input=x_input.reshape(1,n_steps,n_features)
yhat=model1.predict(x_input,verbose=0)
print(yhat)

[[2029895.2]]


In [70]:
#MODEL1 LSTM IS GETTING APPROX PRIDICTION SO MOVING TO stacked lstm
#stacked lstm HAS MULTIPLE HIDDEN LAYERS UNDERSTAND COMLEX PATTERNS AND DEPENDENCIES IN DATA

In [87]:
model2=Sequential()
model2.add(LSTM(50,activation='relu',return_sequences=True,input_shape=(n_steps,n_features)))
model2.add(LSTM(60,activation='relu'))
model2.add(Dense(1))
model2.compile(optimizer='adam',loss='mse')
model2.fit(x,y,epochs=200,verbose=0)

x_input=array([5501340,3422760,3197472])
x_input=x_input.reshape(1,n_steps,n_features)
yhat2=model2.predict(x_input,verbose=0)
print(yhat2)

[[2566890.5]]


In [79]:
#stacked lstm also not predicting well so we are going with BIDIRECTIONAL

In [95]:
x = x.astype(np.float32)
y = y.astype(np.float32)

model3=Sequential()
model3.add(Bidirectional(LSTM(50,activation='relu',input_shape=(n_steps,n_features))))
model3.add(Dense(1))
model3.compile(optimizer='adam',loss='mse')
model3.fit(x,y,epochs=200,verbose=0)


x_input=np.array([5501340,3422760,3197472],dtype=np.float32)
x_input=x_input.reshape(1,n_steps,n_features)
yhat3=model3.predict(x_input,verbose=0)
print(yhat3)

[[2371641.5]]


In [96]:
#Bidirectional LSTM also giving higher than actual so using convolutional

In [99]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import ConvLSTM2D

In [100]:
raw_seq=[2163480,3362496,3247884,5631955,5501340,3422760,3197472,2127160,2002725]
n_steps=4
x,y=split_sequence(raw_seq,n_steps)
for i in range(len(x)):
    print(x[i],y[i])
    


[2163480 3362496 3247884 5631955] 5501340
[3362496 3247884 5631955 5501340] 3422760
[3247884 5631955 5501340 3422760] 3197472
[5631955 5501340 3422760 3197472] 2127160
[5501340 3422760 3197472 2127160] 2002725


In [103]:
n_features=1
n_steps=2
n_seq=2
x=x.reshape((x.shape[0],n_seq,1,n_steps,n_features))
x

array([[[[[2163480],
          [3362496]]],


        [[[3247884],
          [5631955]]]],



       [[[[3362496],
          [3247884]]],


        [[[5631955],
          [5501340]]]],



       [[[[3247884],
          [5631955]]],


        [[[5501340],
          [3422760]]]],



       [[[[5631955],
          [5501340]]],


        [[[3422760],
          [3197472]]]],



       [[[[5501340],
          [3422760]]],


        [[[3197472],
          [2127160]]]]])

In [108]:
model4=Sequential()
model4.add(ConvLSTM2D(filters=64,kernel_size=(1,2),activation='relu',input_shape=(n_seq,1,n_steps,n_features)))
model4.add(Flatten())
model4.add(Dense(1))
model4.compile(optimizer='adam',loss='mse')
model4.fit(x,y,epochs=200,verbose=0)

x_input=array([5631955,5501340,3422760,3197472])
x_input=x_input.reshape((1,n_seq,1,n_steps,n_features))
yhat4=model4.predict(x_input,verbose=0)
print(yhat4)

[[2795696.2]]


In [ ]:
#actual is 2127160